In [1]:
import warnings
import numpy as np
import pandas as pd
from joblib import dump, load
from sklearn.svm import LinearSVR
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, train_test_split
from tensorflow import keras as keras
import tensorflow as tf
from tensorflow.keras.layers import ReLU
warnings.filterwarnings('error','warnings ignored')
random_seed = 1000


In [2]:
# load pretrained models
lsvr_30 = load('LinearSVR\lsvr_30_finetuned.joblib')
lsvr_60 = load('LinearSVR\lsvr_60_finetuned.joblib')
lsvr_90 = load('LinearSVR\lsvr_90_finetuned.joblib')
lsvr_120 = load('LinearSVR\lsvr_120_finetuned.joblib')

sgdr_30 = load('SGDregressor\SGDr_30_finetuned.joblib')
sgdr_60 = load('SGDregressor\SGDr_60_finetuned.joblib')
sgdr_90 = load('SGDregressor\SGDr_90_finetuned.joblib')
sgdr_120 = load('SGDregressor\SGDr_120_finetuned.joblib')

In [3]:

# preprocessing
def preprocess(df,mode = 'train'):
  frac = 0.2
  if mode=='train':
    # fill missing data
    df['Total Cloud Cover [%]'].replace(-7999,np.nan,inplace = True)
    df['Total Cloud Cover [%]'].replace(-6999,np.nan,inplace = True)
    df['Total Cloud Cover [%]'].interpolate(limit = 10,limit_direction = 'both',inplace = True)  

    #  create targets
    df['t_30'] = df.groupby('DATE (MM/DD)')['Total Cloud Cover [%]'].shift(periods = -30,fill_value = -1)
    df['t_60'] = df.groupby('DATE (MM/DD)')['Total Cloud Cover [%]'].shift(periods = -60,fill_value = -1)
    df['t_90'] = df.groupby('DATE (MM/DD)')['Total Cloud Cover [%]'].shift(periods = -90,fill_value = -1)
    df['t_120'] = df.groupby('DATE (MM/DD)')['Total Cloud Cover [%]'].shift(periods = -120,fill_value = -1)

    cond = (df['Total Cloud Cover [%]'] == -1)
    req_samples = df[cond].sample(frac = frac,random_state = random_seed)
    not_req_samples = df[cond].drop(req_samples.index)
    df.drop(not_req_samples.index,inplace=True)
    
    # drop unwanted features
    df.drop([
            'DATE (MM/DD)',
            'MST',
            'Direct sNIP [W/m^2]',                # this feature is highly correlated with cmp22
            'Tower Wet Bulb Temp [deg C]',        # highly correlated with other temperature readings
            'Tower Dew Point Temp [deg C]',
            'Snow Depth [cm]',
            'Moisture',
            'Albedo (CMP11)',
            'Precipitation (Accumulated) [mm]',
            'Azimuth Angle [degrees]'
    ],axis =1,inplace = True)

    return df
  if mode == 'test':
    df['Total Cloud Cover [%]'].replace(-7999,np.nan,inplace = True)
    df['Total Cloud Cover [%]'].replace(-6999,np.nan,inplace = True)
    df['Total Cloud Cover [%]'].interpolate(limit = 10,limit_direction = 'both',inplace = True)  

    df.drop(columns={
      'Time [Mins]',
      'Direct sNIP [W/m^2]',                # this feature is highly correlated with cmp22
      'Tower Wet Bulb Temp [deg C]',        # highly correlated with other temperature readings
      'Tower Dew Point Temp [deg C]',
      'Snow Depth [cm]',
      'Moisture',
      'Albedo (CMP11)',
      'Precipitation (Accumulated) [mm]',
      'Azimuth Angle [degrees]',
      'scenario_set' 
    },inplace = True)

    return df.iloc[-1,]


In [4]:
df = pd.read_csv(r'data\train.csv')
df.head()

,DATE (MM/DD),MST,Global CMP22 (vent/cor) [W/m^2],Direct sNIP [W/m^2],Azimuth Angle [degrees],Tower Dry Bulb Temp [deg C],Tower Wet Bulb Temp [deg C],Tower Dew Point Temp [deg C],Tower RH [%],Total Cloud Cover [%],Peak Wind Speed @ 6ft [m/s],Avg Wind Direction @ 6ft [deg from N],Station Pressure [mBar],Precipitation (Accumulated) [mm],Snow Depth [cm],Moisture,Albedo (CMP11)
0,1/1,00:00,-0.962276,0.000000,356.85640,7.216,0.988,-7.312,32.33,-1,9.95,271.3,806.779,0.0,0.219,0.0,0.0
1,1/1,00:01,-0.937921,0.000000,357.65505,7.251,1.040,-7.260,32.40,-1,8.20,272.9,806.840,0.0,0.206,0.0,0.0
2,1/1,00:02,-0.944395,0.000000,358.45438,7.256,1.093,-7.207,32.54,-1,6.70,288.8,806.876,0.0,0.148,0.0,0.0
3,1/1,00:03,-0.951350,-0.029673,359.25416,7.254,1.060,-7.440,31.89,-1,7.70,294.0,806.823,0.0,0.235,0.0,0.0
4,1/1,00:04,-0.934976,-0.054401,0.05415,7.331,1.081,-7.419,31.78,-1,7.20,285.5,806.762,0.0,0.182,0.0,0.0


In [5]:
df = preprocess(df,mode='train')
df.describe()

,Global CMP22 (vent/cor) [W/m^2],Tower Dry Bulb Temp [deg C],Tower RH [%],Total Cloud Cover [%],Peak Wind Speed @ 6ft [m/s],Avg Wind Direction @ 6ft [deg from N],Station Pressure [mBar],t_30,t_60,t_90,t_120
count,307216.000000,307216.000000,307216.000000,307216.000000,307216.000000,307216.000000,307216.000000,307216.000000,307216.000000,307216.000000,307216.000000
mean,338.139172,14.156688,37.238903,39.764552,2.993360,146.297582,816.845133,39.047805,37.940785,36.795175,35.622423
std,311.606434,10.861875,23.357442,37.528009,2.072404,108.565210,5.057391,37.804043,37.948477,38.023448,38.060778
min,-4.764200,-16.440000,4.210000,-1.000000,0.000000,0.000000,795.065000,-1.000000,-1.000000,-1.000000,-1.000000
25%,54.990850,5.446000,19.760000,7.000000,1.700000,43.020000,813.761000,6.000000,4.000000,-1.000000,-1.000000
50%,261.436500,15.040000,30.430000,23.000000,2.700000,122.000000,817.252000,22.000000,20.000000,19.000000,18.000000
75%,551.931250,23.040000,47.930000,81.000000,3.950000,270.600000,820.268000,81.000000,79.000000,78.000000,76.000000
max,1428.650000,36.320000,100.100000,100.000000,23.450000,360.000000,847.963000,100.000000,100.000000,100.000000,100.000000


In [6]:
df['Total Cloud Cover [%]'].value_counts(normalize=True)

-1.000000     0.178884
 99.000000    0.034569
 98.000000    0.031209
 97.000000    0.029439
 96.000000    0.025035
                ...   
 47.750000    0.000003
 65.500000    0.000003
 28.250000    0.000003
 15.857143    0.000003
 80.750000    0.000003
Name: Total Cloud Cover [%], Length: 353, dtype: float64

In [7]:
df.shape

(307216, 11)

In [8]:
X_30,Y_30 = df.iloc[:,:-4].values,df['t_30'].values
X_60,Y_60 = df.iloc[:,:-3].values,df['t_60'].values
X_90,Y_90= df.iloc[:,:-2].values,df['t_90'].values
X_120,Y_120 = df.iloc[:,:-1].values,df['t_120'].values 

In [9]:
def split(x,y,train_size=0.70):
  return train_test_split(x,y,train_size=train_size,random_state=random_seed)

In [10]:
def create_and_fit_nn(x,y,nn=None,predict=False):
    if predict == False:
        nn = tf.keras.Sequential()
        nn.add(tf.keras.layers.Dense(4, input_shape=(2,)))
        nn.add(ReLU(max_value=100,threshold=-1.0)) 
        nn.add(tf.keras.layers.Dense(1))
        # early stopping
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',restore_best_weights=True, patience=5)

        nn.compile(
            optimizer='sgd',
            loss=tf.keras.losses.MeanAbsoluteError(),
            metrics=[tf.keras.metrics.MeanAbsoluteError()])
        # print(nn.summary())

        history = nn.fit(x, y,validation_split = 0.3,verbose = 1, epochs=50,callbacks = [callback])
        print(min(history.history['loss']))
        print(min(history.history['val_loss']))
        return nn

    if predict ==True:
        nn_preds = nn.predict(x)
        score = mae(y,nn_preds)
        nn.evaluate(x, y, verbose=2)
        return score

In [11]:
def run(lsvr,sgd,x,y,nn=None,mode = 'train'):
    # split data
    x_train,x_test,y_train,y_test = split(x,y)
    
    if mode == 'train':
        lsvr_preds = lsvr.predict(x_train)
        sgd_preds = sgd.predict(x_train)
        inputs = np.column_stack((lsvr_preds,sgd_preds))
        nn = create_and_fit_nn(inputs,y_train,predict=False)
        print(f'--------training finished--------')
        return nn
    if mode == 'test':
        lsvr_preds = lsvr.predict(x_test)
        sgd_preds = sgd.predict(x_test)
        inputs = np.column_stack((lsvr_preds,sgd_preds))
        mae_score = create_and_fit_nn(inputs,y_test,nn,predict=True)
        print('mae score:', mae_score)

In [12]:
nn_30 = run(lsvr_30,sgdr_30,X_30,Y_30,mode='train')
run(lsvr_30,sgdr_30,X_30,Y_30,nn = nn_30,mode='test')

nn_60 = run(lsvr_60,sgdr_60,X_60,Y_60,mode='train')
run(lsvr_60,sgdr_60,X_60,Y_60,nn = nn_60,mode='test')

nn_90 = run(lsvr_90,sgdr_90,X_90,Y_90,mode='train')
run(lsvr_90,sgdr_90,X_90,Y_90,nn = nn_90,mode='test')

nn_120 = run(lsvr_120,sgdr_120,X_120,Y_120,mode='train')
run(lsvr_120,sgdr_120,X_120,Y_120,nn = nn_120,mode='test')

Epoch 1/50
4705/4705 [==============================] - 11s 2ms/step - loss: 32.3539 - mean_absolute_error: 32.3539 - val_loss: 28.9631 - val_mean_absolute_error: 28.9631
Epoch 2/50
4705/4705 [==============================] - 9s 2ms/step - loss: 28.2602 - mean_absolute_error: 28.2602 - val_loss: 27.8035 - val_mean_absolute_error: 27.8035
Epoch 3/50
4705/4705 [==============================] - 8s 2ms/step - loss: 27.3810 - mean_absolute_error: 27.3810 - val_loss: 27.1120 - val_mean_absolute_error: 27.1120
Epoch 4/50
4705/4705 [==============================] - 8s 2ms/step - loss: 26.2954 - mean_absolute_error: 26.2954 - val_loss: 26.3653 - val_mean_absolute_error: 26.3653
Epoch 5/50
4705/4705 [==============================] - 8s 2ms/step - loss: 23.5662 - mean_absolute_error: 23.5662 - val_loss: 20.6034 - val_mean_absolute_error: 20.6034
Epoch 6/50
4705/4705 [==============================] - 9s 2ms/step - loss: 23.4205 - mean_absolute_error: 23.4205 - val_loss: 18.1654 - val_mean_abs

In [13]:
def ensemble_preds(lsvr,sgd,nn,x):
    lsvr_preds = lsvr.predict(x.values.reshape(1,-1))
    sgd_preds = sgd.predict(x.values.reshape(1,-1))
    inputs = np.column_stack((lsvr_preds,sgd_preds))
    nn_preds = nn.predict(inputs)
    return nn_preds

In [ ]:

wd_test = pd.read_csv(r'data\shell_test.csv')
test = pd.read_csv(r'data\test.csv')

file_count = 0
for set in range(1,301):
  wd = wd_test[wd_test['scenario_set'] == set]
  # preprocessing test data
  last_sample = preprocess(wd,mode='test')

  # predicting test samples
  pred_30 = ensemble_preds(lsvr_30,sgdr_30,nn_30,last_sample)
  last_sample['t_30'] = pred_30.item()
  pred_60 = ensemble_preds(lsvr_60,sgdr_60,nn_60,last_sample)
  last_sample['t_60'] = pred_60.item()
  pred_90 = ensemble_preds(lsvr_90,sgdr_90,nn_90,last_sample)
  last_sample['t_90'] = pred_90.item()
  pred_120 = ensemble_preds(lsvr_120,sgdr_120,nn_120,last_sample)
  
  # fill in test data using above predictions
  test.iloc[set-1,test.columns.get_indexer(['30_min_horizon'])] = np.round(pred_30.item())
  test.iloc[set-1,test.columns.get_indexer(['60_min_horizon'])] = np.round(pred_60.item())
  test.iloc[set-1,test.columns.get_indexer(['90_min_horizon'])] = np.round(pred_90.item())
  test.iloc[set-1,test.columns.get_indexer(['120_min_horizon'])] = np.round(pred_120.item())

  file_count += 1
  if file_count%30 == 0 :
      print(file_count)

In [15]:
test = test.applymap(int)
test.to_csv('lsvr_sgd_preds.csv',index=False)

print(test.describe())
test

       scenario_set  30_min_horizon  60_min_horizon  90_min_horizon  \
count    300.000000      300.000000      300.000000      300.000000   
mean     150.500000       49.930000       50.823333       58.073333   
std       86.746758       28.731925       29.290132       33.776780   
min        1.000000        3.000000        1.000000       -1.000000   
25%       75.750000       18.000000       18.000000       22.000000   
50%      150.500000       55.000000       59.000000       69.000000   
75%      225.250000       79.000000       78.250000       88.250000   
max      300.000000       82.000000       84.000000       95.000000   

       120_min_horizon  
count       300.000000  
mean         57.126667  
std          32.869418  
min          -1.000000  
25%          24.000000  
50%          70.000000  
75%          85.000000  
max          92.000000  


,scenario_set,30_min_horizon,60_min_horizon,90_min_horizon,120_min_horizon
0,1,81,81,91,88
1,2,41,47,60,65
2,3,82,83,94,90
3,4,52,56,68,69
4,5,20,23,29,34
...,...,...,...,...,...
295,296,7,4,1,-1
296,297,53,57,67,67
297,298,9,6,2,-1
298,299,37,40,48,51
